In [1]:
%load_ext autoreload
%autoreload 2
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=0

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=0


In [2]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm.autonotebook import tqdm
import pandas as pd
from cuml.manifold.umap import UMAP as cumlUMAP
from avgn.utils.paths import DATA_DIR, most_recent_subdirectory, ensure_dir
from avgn.signalprocessing.create_spectrogram_dataset import flatten_spectrograms

/mnt/cube/tsainbur/conda_envs/tpy3/lib/python3.6/site-packages/tqdm/autonotebook/__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


### load data

In [3]:
DATASET_ID = 'koumura_bengalese_finch'
df_loc =  DATA_DIR / 'syllable_dfs' / DATASET_ID / 'bf.pickle'

In [4]:
syllable_df = pd.read_pickle(df_loc)

In [5]:
syllable_df[:3]

,start_time,end_time,labels,sequence_num,indv,indvi,key,rate,spectrogram
0,1.158,1.249,0,0,Bird3,0,Bird3_322,32000,"[[0, 0, 0, 110, 190, 203, 216, 217, 216, 219, ..."
1,1.323,1.404,0,0,Bird3,0,Bird3_322,32000,"[[0, 0, 0, 54, 164, 182, 187, 191, 196, 196, 1..."
2,1.530,1.564,0,0,Bird3,0,Bird3_322,32000,"[[0, 0, 0, 0, 0, 106, 100, 181, 209, 202, 196,..."


In [6]:
np.shape(syllable_df.spectrogram.values[0])

(32, 21)

### project

In [7]:
ensure_dir(DATA_DIR / 'embeddings' / DATASET_ID / 'full')

In [8]:
for indv in tqdm(syllable_df.indv.unique()):
    subset_df = syllable_df[syllable_df.indv == indv]
    
    specs = list(subset_df.spectrogram.values)
    specs = [i/np.max(i) for i in tqdm(specs)]
    specs_flattened = flatten_spectrograms(specs)
    print(np.shape(specs_flattened))
    
    cuml_umap = cumlUMAP(0.01)
    embedding = cuml_umap.fit_transform(specs_flattened)
    subset_df['umap-0.01'] = list(embedding)
    
    cuml_umap = cumlUMAP(min_dist=0.1)
    embedding = cuml_umap.fit_transform(specs_flattened)
    subset_df['umap-0.1'] = list(embedding)
    
    cuml_umap = cumlUMAP(min_dist=0.5)
    embedding = cuml_umap.fit_transform(specs_flattened)
    subset_df['umap-0.5'] = list(embedding)
    
    ensure_dir(DATA_DIR / 'embeddings' / 'multiple_spread' / DATASET_ID / 'indvs')
    subset_df.to_pickle(DATA_DIR / 'embeddings' / 'multiple_spread' / DATASET_ID / 'indvs' / (indv + '.pickle'))

(29470, 672)



Exception ignored in: <bound method UMAP.__del__ of UMAP(n_neighbors=0.01, verbose=False, handle=<cuml.common.handle.Handle object at 0x7f05e6a78df0>, hash_input=False)>
Traceback (most recent call last):
  File "cuml/manifold/umap.pyx", line 394, in cuml.manifold.umap.UMAP.__del__
TypeError: an integer is required


RuntimeError: Exception occured! file=/conda/conda-bld/libcuml_1583952676191/work/cpp/src_prims/linalg/binary_op.h line=52: FAIL: call='cudaPeekAtLastError()'. Reason:no kernel image is available for execution on the device

Obtained 64 stack frames
#0 in /mnt/cube/tsainbur/conda_envs/tpy3/lib/python3.6/site-packages/cuml/utils/pointer_utils.cpython-36m-x86_64-linux-gnu.so(_ZN8MLCommon9Exception16collectCallStackEv+0x3e) [0x7f053ac4e37e]
#1 in /mnt/cube/tsainbur/conda_envs/tpy3/lib/python3.6/site-packages/cuml/utils/pointer_utils.cpython-36m-x86_64-linux-gnu.so(_ZN8MLCommon9ExceptionC1ERKNSt7__cxx1112basic_stringIcSt11char_traitsIcESaIcEEE+0x80) [0x7f053ac4ee90]
#2 in /mnt/cube/tsainbur/conda_envs/tpy3/lib/python3.6/site-packages/cuml/utils/../../../../libcuml++.so(_ZN8MLCommon6LinAlg12binaryOpImplIfLi4E17__nv_dl_wrapper_tI11__nv_dl_tagIPFvPfPKfS6_iP11CUstream_stEXadL_ZNS0_10eltwiseSubIfiEEvPT_PKSC_SF_T0_S8_EELj1EEIEEifLi256EEEvPT3_SF_SF_T2_T1_S8_+0x27a) [0x7f0529cef5ba]
#3 in /mnt/cub